<a href="https://colab.research.google.com/github/aggelospsiris/Data-mining-and-Learning-Algorithms/blob/main/Random_Forest/Tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
import pandas as pd

# Read csv file
df = pd.read_csv('/content/sample_data/amazon.csv')
 
# View the first 5 rows of data
print(df.head())
 
# Tokenize each item in the review column
word_tokens = [word_tokenize(text) for text in df.Text]

                                                Text  Score
0  The description and photo on this product need...      3
1  This was a great book!!!! It is well thought t...      5
2  I am a first year teacher, teaching 5th grade....      5
3  I got the book at my bookfair at school lookin...      5
4  Hi! I'm Martine Redman and I created this puzz...      5


Note that we tokenize the reviews in a list comprehension. List comprehensions in python can make use of vectorizing loops and hence provide faster computation than a Python for loop.

Next, we add another column to the data frame called ”n_words’, which contains the number of words in the review. We do this because we believe that the number of words in a review can potentially have predictive power on such a classification problem.

In [22]:
# Create an empty list to store the length of the reviews
len_tokens = []
 
# Iterate over the word_tokens list and determine the length of each item
for i in range(len(word_tokens)):
    len_tokens.append(len(word_tokens[i]))
 
# Create a new feature for the lengh of each review
df['n_words'] = len_tokens


I am implementing the tfidf method to transform the reviews into numeric data

In [23]:
# Build the vectorizer
vect = TfidfVectorizer(
                stop_words=ENGLISH_STOP_WORDS,
                ngram_range=(1, 2),
                max_features=200,
                token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(df.Text)
 
# Create sparse matrix from the vectorizer
X = vect.transform(df.Text)



In [28]:
# Create a DataFrame
df2 = pd.DataFrame(X.toarray(), columns=vect.get_feature_names())
df2['y'] = df.Score



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [31]:
import numpy as np

# Transform data into predictors 'X' and outcome 'y'
X = df2.drop('y', axis=1).values
y = df2['y'].values
 
unique, counts = np.unique(y, return_counts=True)
print('Total number of cases: %d'%len(y))
print('Number of cases in each classification: ', dict(zip(unique, counts)))




Total number of cases: 50000
Number of cases in each classification:  {1: 5021, 2: 2918, 3: 3902, 4: 8384, 5: 29775}


In [35]:
from sklearn.model_selection import train_test_split
 
# Split dataset into 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size=0.2,
                                                      stratify=y,
                                                      random_state=100)



In [36]:
#Random Forest model
from sklearn.ensemble import RandomForestClassifier
 
# Instantiate Random Forest Classifier
model = RandomForestClassifier(n_estimators=800,
                                   max_depth=4,
                                   min_samples_leaf=0.001,
                                   random_state=100,
                                   n_jobs=-1)
 
# Fit model to the training set
model.fit(X_train, y_train)



RandomForestClassifier(max_depth=4, min_samples_leaf=0.001, n_estimators=800,
                       n_jobs=-1, random_state=100)

In [39]:
#Metrics

from sklearn import metrics
 
# Predict test set labels
y_pred = model.predict(X_test)

# Evaluate test set accuracy
print('Accuracy score (train set) = %0.4f'%metrics.accuracy_score(y_train, model.predict(X_train)))
print('Accuracy score (test set) = %0.4f'%metrics.accuracy_score(y_test, y_pred))



 
print(metrics.classification_report(y_test, y_pred))



Accuracy score (train set) = 0.5955
Accuracy score (test set) = 0.5955
              precision    recall  f1-score   support

           1       0.00      0.00      0.00      1004
           2       0.00      0.00      0.00       584
           3       0.00      0.00      0.00       780
           4       0.00      0.00      0.00      1677
           5       0.60      1.00      0.75      5955

    accuracy                           0.60     10000
   macro avg       0.12      0.20      0.15     10000
weighted avg       0.35      0.60      0.44     10000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
